In [ ]:
##############
## Script listens to serial port and writes contents into a file
##############
## requires pySerial to be installed 
import serial
import time
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
import math
from keras.utils import np_utils

from __future__ import print_function

import keras
import keras.callbacks as cb
from keras.datasets import mnist
from keras.layers.core import Activation, Dense, Dropout
from keras.models import Sequential 
from keras.optimizers import SGD
from keras.regularizers import l1, l2

from sklearn import preprocessing
import pickle


In [2]:
def dataProcess(file,startCutThreshold,rangeThreshold,offsetNoise):
    '''startCutThreshold: cut noise in the begining (500)
       rangeThreshold:find maximum peak value in this range (80)
       offsetNoise:noise level above average (20)
    '''
    list_of_lists = []
    with open(file, "r") as f:
        for line in f:
            inner_list = [elt.strip() for elt in line.split('\t')]
            # in alternative, if you need to use the file content as numbers
            # inner_list = [int(elt.strip()) for elt in line.split(',')]
            list_of_lists.append(inner_list)
    
        
    a = np.shape(list_of_lists)
    a = a[0]
    data = np.zeros((int(a/2),10))

    for x in range(0,a,2):
        data[int(x/2)] = list(map(int,list_of_lists[x]))


    '''cut noise in the begining: 500'''
    data = data[startCutThreshold:,:]
    
    '''find channel dominate'''
    index_max = np.argmax(data[:,1:])
    channelDominate = index_max%9+1
    print(channelDominate)
        
    plt.plot(data[:,0],data[:,channelDominate],linewidth=0.5)
    plt.suptitle('Channel Dominate')
    plt.show()
    
    '''find peaks'''
    plt.plot(data[:,0],data[:,channelDominate],linewidth=0.5)
    peakind = signal.find_peaks_cwt(data[:,channelDominate],np.arange(1,300))
    plt.plot(data[peakind,0], data[peakind,channelDominate], 'rx')
    plt.suptitle('Raw Peaks')
    plt.show()
    
    '''find accurate peaks'''
    peakind0 = accPeak(peakind,data,channelDominate,rangeThreshold)
    plt.plot(data[:,0],data[:,channelDominate],linewidth=0.5)
    plt.plot(data[peakind0,0], data[peakind0,channelDominate], 'rx') 
    plt.suptitle('Accurate Peaks')
    plt.show()
        
    '''filter noise'''
    ind = filterNoise(peakind0,data,channelDominate,offsetNoise)
    peakind0 = np.delete(peakind0,ind,None)
    plt.plot(data[:,0],data[:,channelDominate],linewidth=0.5)
    plt.plot(data[peakind0,0], data[peakind0,channelDominate], 'rx') 
    plt.suptitle('Filter Noise')
    plt.show()
    
    '''filter 10% highest and 10% lowest peaks'''
    ind = filterPeck(peakind0,data,channelDominate)
    peakind0 = np.delete(peakind0,ind,None)
    plt.plot(data[:,0],data[:,channelDominate],linewidth=0.5)
    plt.plot(data[peakind0,0], data[peakind0,channelDominate], 'rx')
    plt.suptitle('Filter High & Low')
    plt.show()
    
    peakindTrue = []
    for channel in range(1, 10):
        peakindTrue[channel-1] = accPeak(peakind0,data,channel,rangeThreshold)
        plt.plot(data[:,0],data[:,channel],linewidth=0.5)
        plt.plot(data[peakindTrue[channel-1],0], data[peakindTrue[channel-1],channel], 'rx') 
        plt.suptitle('sensor'+str(channel))
        plt.show()
  
    dataProcessed = []
    for i in range(1, 10):
        dataProcessed.append(data[peakindTrue[i-1], i])

    dataProcessed = list(map(list,zip(*dataProcessed)))
    return dataProcessed

def filterNoise(peakind,data,channel,offsetNoise):
    ind = []
    noise = np.mean(data[:,channel])
    for i in range(len(peakind)):
        if data[peakind,channel][i] < noise +offsetNoise:
            ind.append(i)
    return ind

def filterPeck(peakind,data,channel):
    ind = []
    dataPeak = []
    cutNumber = math.ceil(0.1*len(peakind))
    for i in range(len(peakind)):
        dataPeak.append(data[peakind,channel][i])
        
    indexSorted = sorted(enumerate(dataPeak), key=lambda x:x[1])
    for i in range(cutNumber):
        ind.append(indexSorted[i][0])
        ind.append(indexSorted[len(peakind)-1-i][0])
   
    return ind

def accPeak(peakind,data,channel,rangeThreshold):
    peakindTrue = []
    for j in range(peakind.shape[0]):
        maxvalue = data[peakind[j],channel]
        maxind = peakind[j]
        for k in range(max(-rangeThreshold,-peakind[j]),min(rangeThreshold+1,data.shape[0]-1-peakind[j])):
            if data[peakind[j]+k,channel]>maxvalue:
                maxvalue = data[peakind[j]+k,channel]
                maxind = peakind[j]+k
        peakindTrue.append(maxind)
        
    return peakindTrue

In [29]:
write_to_file_path = "C:\\Users\\PangTouXian\\Desktop\\gao project\\experiment\\train3\\1.txt";

In [32]:
output_file = open(write_to_file_path, "w+")
ser = serial.Serial('COM3',250000)
line = ser.readline()
line = ser.readline()
line = ser.readline()
line = ser.readline()
num = 1
while num<5000:
    line = ser.readline()
    line = line.decode("ISO-8859-1") #ser.readline returns a binary, convert to string
    #print(line)
    output_file.write(line)
    num = num+1

In [ ]:
test0 = dataProcess(write_to_file_path,500,80,50)

'''position detection'''
for i in range(np.shape(test0)[0]):
    result1 = TestModel1(model=trained_model1, data=test0[i])
    print(result1[12][0]+1,result1[11][0]+1)

'''force amplitude detection'''
for i in range(np.shape(test0)[0]):
    result1 = TestModel2(model1=trained_model1,model2=trained_model2, data=test0[i])
    print(result1[2][0]+1)